# HydroLearn Teaching Quickstart Notebook
# HydroLearn 教学快速入门笔记本

This notebook demonstrates the HBV and SCS-CN+UH hydrological models included in HydroLearn.

本笔记本演示HydroLearn中包含的HBV和SCS-CN+UH水文模型。

## Learning Objectives / 学习目标

1. Understand the HBV continuous simulation model / 理解HBV连续模拟模型
2. Understand the SCS-CN event-based model / 理解基于事件的SCS-CN模型
3. Compare continuous vs. event-based modeling approaches / 比较连续与事件模型方法
4. Visualize model components and outputs / 可视化模型组件和输出

In [ ]:
# Import required libraries / 导入所需库
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import sys
import os

# Add parent directory to path to import models / 添加父目录到路径以导入模型
sys.path.insert(0, os.path.abspath('..'))

# Import HydroLearn models / 导入HydroLearn模型
from hbv_model import HBVModel, calculate_nse
from event_model_scs_uh import SCS_UH_EventModel, calculate_peak_flow

# Set style for better visualizations / 设置样式以获得更好的可视化
plt.style.use('seaborn-v0_8-darkgrid')
np.random.seed(42)  # For reproducibility / 用于可重现性

print("Libraries imported successfully! / 库导入成功！")

## Part 1: HBV Continuous Simulation Model
## 第一部分：HBV连续模拟模型

The HBV model is a conceptual rainfall-runoff model that simulates the complete hydrological cycle.

HBV模型是一个概念性降雨-径流模型，模拟完整的水文循环。

### Key Components / 关键组件:
- Snow routine (accumulation & melt) / 积雪程序（累积和融化）
- Soil moisture accounting / 土壤水分核算
- Response function (upper & lower reservoirs) / 响应函数（上层和下层水库）

In [ ]:
# Generate synthetic meteorological data (1 year) / 生成合成气象数据（1年）
n_days = 365
dates = [datetime(2020, 1, 1) + timedelta(days=i) for i in range(n_days)]

# Synthetic precipitation (mm/day) with seasonal variation / 具有季节变化的合成降水
P = np.random.gamma(2, 3, n_days)
seasonal = np.sin(2 * np.pi * np.arange(n_days) / 365) * 2 + 3
P = P * seasonal

# Synthetic temperature (°C) with seasonal variation / 具有季节变化的合成温度
T = 10 + 15 * np.sin(2 * np.pi * (np.arange(n_days) - 80) / 365) + np.random.normal(0, 2, n_days)

# Synthetic potential ET (mm/day) / 合成潜在蒸散发
Ep = 2 + 3 * np.sin(2 * np.pi * (np.arange(n_days) - 80) / 365) + np.random.normal(0, 0.3, n_days)
Ep = np.maximum(Ep, 0)

print(f"Generated {n_days} days of synthetic data / 生成了{n_days}天的合成数据")
print(f"Mean precipitation / 平均降水: {np.mean(P):.2f} mm/day")
print(f"Mean temperature / 平均温度: {np.mean(T):.2f} °C")
print(f"Mean potential ET / 平均潜在蒸散发: {np.mean(Ep):.2f} mm/day")

In [ ]:
# Initialize HBV model with default parameters / 使用默认参数初始化HBV模型
hbv = HBVModel(
    TT=0.0,      # Threshold temperature / 阈值温度
    CFMAX=3.5,   # Degree-day factor / 度日因子
    FC=250.0,    # Field capacity / 田间持水量
    LP=0.7,      # Limit for potential ET / 潜在蒸散发限制
    BETA=2.0,    # Shape coefficient / 形状系数
    K0=0.3,      # Fast flow recession / 快速流消退
    K1=0.05,     # Slow flow recession / 慢速流消退
    K2=0.01,     # Baseflow recession / 基流消退
    UZL=20.0,    # Threshold for fast response / 快速响应阈值
    PERC=2.0     # Max percolation rate / 最大下渗率
)

print("HBV model initialized! / HBV模型已初始化！")

In [ ]:
# Run HBV simulation / 运行HBV模拟
results_hbv = hbv.run(P, T, Ep, warmup=90)

# Extract results / 提取结果
Q_hbv = results_hbv['Q']
Q_valid = Q_hbv[~np.isnan(Q_hbv)]

print("HBV simulation completed! / HBV模拟完成！")
print(f"Mean discharge / 平均流量: {np.mean(Q_valid):.2f} mm/day")
print(f"Peak discharge / 峰值流量: {np.max(Q_valid):.2f} mm/day")

In [ ]:
# Visualize HBV results / 可视化HBV结果
fig, axes = plt.subplots(3, 1, figsize=(14, 10))
fig.suptitle('HBV Model Simulation Results / HBV模型模拟结果', fontsize=14, fontweight='bold')

# Plot 1: Meteorological inputs / 气象输入
ax1 = axes[0]
ax1_twin = ax1.twinx()
ax1.bar(dates, P, alpha=0.6, color='blue', label='Precipitation / 降水')
ax1_twin.plot(dates, T, color='red', linewidth=1, label='Temperature / 温度')
ax1.set_ylabel('Precipitation (mm/day)', color='blue')
ax1_twin.set_ylabel('Temperature (°C)', color='red')
ax1.set_title('Meteorological Inputs / 气象输入')
ax1.grid(True, alpha=0.3)

# Plot 2: Discharge components / 径流组分
ax2 = axes[1]
ax2.fill_between(dates, 0, results_hbv['Q2'], alpha=0.5, color='green', label='Baseflow / 基流')
ax2.fill_between(dates, results_hbv['Q2'], results_hbv['Q2'] + results_hbv['Q1'], 
                 alpha=0.5, color='orange', label='Slow flow / 慢速流')
ax2.fill_between(dates, results_hbv['Q2'] + results_hbv['Q1'], results_hbv['Q'], 
                 alpha=0.5, color='red', label='Fast flow / 快速流')
ax2.set_ylabel('Discharge (mm/day)')
ax2.set_title('Discharge Components / 径流组分')
ax2.legend(loc='upper right')
ax2.grid(True, alpha=0.3)

# Plot 3: Soil moisture and snow / 土壤水分和积雪
ax3 = axes[2]
ax3_twin = ax3.twinx()
ax3.plot(dates, results_hbv['SM'], color='brown', linewidth=1.5, label='Soil Moisture / 土壤水分')
ax3.axhline(y=hbv.FC, color='brown', linestyle='--', alpha=0.5, label='Field Capacity / 田间持水量')
ax3_twin.fill_between(dates, 0, results_hbv['SNOW'], alpha=0.4, color='lightblue', label='Snow / 积雪')
ax3.set_ylabel('Soil Moisture (mm)', color='brown')
ax3_twin.set_ylabel('Snow Storage (mm)', color='lightblue')
ax3.set_xlabel('Date / 日期')
ax3.set_title('Soil Moisture and Snow / 土壤水分和积雪')
ax3.legend(loc='upper left')
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Visualization complete! / 可视化完成！")

## Part 2: SCS-CN Event Model
## 第二部分：SCS-CN事件模型

The SCS Curve Number method is an event-based model for estimating direct runoff from storm events.

SCS曲线数法是一种基于事件的模型，用于估算暴雨事件的直接径流。

### Key Features / 关键特征:
- Event-based (single storm) / 基于事件（单次暴雨）
- Curve Number represents watershed characteristics / 曲线数代表流域特征
- Triangular unit hydrograph for routing / 三角形单位线进行汇流

In [ ]:
# Define catchment parameters for SCS-CN model / 定义SCS-CN模型的流域参数
CN = 75.0          # Curve Number / 曲线数
area = 50.0        # km² / 平方公里
Tc = 4.0           # Time of concentration (hours) / 汇流时间（小时）
dt = 1.0           # Time step (hours) / 时间步长（小时）

print(f"Catchment Parameters / 流域参数:")
print(f"  Curve Number / 曲线数: {CN}")
print(f"  Area / 面积: {area} km²")
print(f"  Time of Concentration / 汇流时间: {Tc} hours")

In [ ]:
# Create synthetic storm event (24 hours) / 创建合成暴雨事件（24小时）
n_hours = 24
rainfall = np.zeros(n_hours)

# Concentrated rainfall in hours 6-12 / 集中降雨在6-12小时
storm_start = 6
storm_duration = 6
peak_hour = storm_start + storm_duration // 2

for i in range(storm_start, storm_start + storm_duration):
    distance_from_peak = abs(i - peak_hour)
    rainfall[i] = 20.0 * np.exp(-0.5 * (distance_from_peak / 1.5) ** 2)

print(f"Storm Characteristics / 暴雨特征:")
print(f"  Total Rainfall / 总降雨量: {np.sum(rainfall):.1f} mm")
print(f"  Peak Intensity / 峰值强度: {np.max(rainfall):.1f} mm/hour")
print(f"  Storm Duration / 暴雨历时: {storm_duration} hours")

In [ ]:
# Initialize and run SCS-CN + UH model / 初始化并运行SCS-CN + UH模型
scs_model = SCS_UH_EventModel(CN=CN, area=area, Tc=Tc, dt=dt)
results_scs = scs_model.run(rainfall)

# Extract results / 提取结果
runoff = results_scs['runoff']
streamflow = results_scs['streamflow']

# Calculate statistics / 计算统计量
total_runoff = np.sum(runoff)
runoff_coef = total_runoff / np.sum(rainfall) if np.sum(rainfall) > 0 else 0
peak_flow, peak_time_idx = calculate_peak_flow(streamflow)
peak_time = peak_time_idx * dt

print("\nSimulation Results / 模拟结果:")
print(f"  Total Runoff / 总径流量: {total_runoff:.1f} mm")
print(f"  Runoff Coefficient / 径流系数: {runoff_coef:.2f}")
print(f"  Peak Flow / 峰值流量: {peak_flow:.2f} m³/s")
print(f"  Time to Peak / 峰现时间: {peak_time:.1f} hours")

In [ ]:
# Visualize SCS-CN results / 可视化SCS-CN结果
fig, axes = plt.subplots(3, 1, figsize=(12, 10))
fig.suptitle('SCS-CN + UH Event Model / SCS-CN + UH事件模型', fontsize=14, fontweight='bold')

# Plot 1: Rainfall / 降雨
ax1 = axes[0]
time_rain = results_scs['time_rainfall']
ax1.bar(time_rain, rainfall, width=dt*0.8, alpha=0.7, color='blue', edgecolor='darkblue')
ax1.set_ylabel('Rainfall (mm/hour)')
ax1.set_title('Rainfall Hyetograph / 降雨过程线')
ax1.grid(True, alpha=0.3)

# Plot 2: Direct runoff / 直接径流
ax2 = axes[1]
ax2.bar(time_rain, runoff, width=dt*0.8, alpha=0.7, color='green', edgecolor='darkgreen')
ax2.set_ylabel('Direct Runoff (mm/hour)')
ax2.set_title('Direct Runoff / 直接径流')
ax2.grid(True, alpha=0.3)

# Plot 3: Streamflow / 流量
ax3 = axes[2]
time_flow = results_scs['time_streamflow']
ax3.plot(time_flow, streamflow, linewidth=2.5, color='red', label='Streamflow')
ax3.fill_between(time_flow, 0, streamflow, alpha=0.3, color='red')
ax3.scatter([peak_time], [peak_flow], color='darkred', s=100, zorder=5, 
            label=f'Peak: {peak_flow:.1f} m³/s')
ax3.set_ylabel('Streamflow (m³/s)')
ax3.set_xlabel('Time (hours) / 时间（小时）')
ax3.set_title('Streamflow Hydrograph / 流量过程线')
ax3.legend(loc='upper right')
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Visualization complete! / 可视化完成！")

## Part 3: Comparison of Models
## 第三部分：模型比较

### HBV vs. SCS-CN Model Comparison
### HBV与SCS-CN模型比较

| Feature | HBV Model | SCS-CN Model |
|---------|-----------|-------------|
| Type / 类型 | Continuous simulation / 连续模拟 | Event-based / 基于事件 |
| Time scale / 时间尺度 | Daily to hourly / 日到小时 | Hourly (storm events) / 小时（暴雨事件） |
| Snow routine / 积雪程序 | Yes / 有 | No / 无 |
| Soil moisture / 土壤水分 | Detailed accounting / 详细核算 | Simple CN-based / 简单基于CN |
| Baseflow / 基流 | Yes (multiple reservoirs) / 有（多个水库） | No / 无 |
| Best for / 最适用于 | Long-term simulation / 长期模拟 | Flood event analysis / 洪水事件分析 |
| Parameters / 参数 | 10 parameters / 10个参数 | 3 parameters (CN, area, Tc) / 3个参数 |
| Data requirements / 数据需求 | P, T, PET / 降水、温度、潜在蒸散发 | Rainfall only / 仅降雨 |

## Exercises / 练习

### Exercise 1 / 练习1:
Try changing the HBV parameters (e.g., FC, BETA, K1) and observe how the discharge changes.

尝试更改HBV参数（例如FC、BETA、K1）并观察流量如何变化。

### Exercise 2 / 练习2:
Modify the Curve Number (CN) in the SCS-CN model and see how it affects the runoff coefficient.

修改SCS-CN模型中的曲线数（CN）并查看它如何影响径流系数。

### Exercise 3 / 练习3:
Create a larger storm event (higher intensity, longer duration) and analyze the resulting hydrograph.

创建一个更大的暴雨事件（更高强度、更长持续时间）并分析生成的过程线。

In [ ]:
# Your code for exercises here / 在此处编写练习代码


## Summary / 总结

In this notebook, you learned:

在本笔记本中，您学到了：

1. How to use the HBV model for continuous hydrological simulation
   如何使用HBV模型进行连续水文模拟

2. How to use the SCS-CN method for event-based runoff estimation
   如何使用SCS-CN方法进行基于事件的径流估算

3. The differences between continuous and event-based modeling
   连续和基于事件建模之间的差异

4. How to visualize and interpret model outputs
   如何可视化和解释模型输出

### Next Steps / 下一步:
- Explore other models in HydroLearn (Xinanjiang, Tank, GR4J, Sacramento)
  探索HydroLearn中的其他模型（新安江、Tank、GR4J、Sacramento）
- Try calibrating models with real data using `calibration_example.py`
  尝试使用`calibration_example.py`用真实数据率定模型
- Compare model performances on the same dataset
  在相同数据集上比较模型性能